# 酒駕判決書文字資料清理與結構化

這次專題基於「民眾對於酒駕者的判刑度不理解，社會輿論缺乏針對酒駕修法的背景脈絡」出發，搜集了大量的酒駕判決書，但判決書以文字寫成，刑度、罰金或其他背景分析，都需要經過資料清理與轉化，才能進一步統計。

這次使用的判決書總數量達49萬份，若以人力編碼曠日廢時，且恐面臨編碼員信度考驗。這次我們以正規表達式（regular expression）鎖定文字特徵萃取，以程式掃描文字、輔以人工抽查，以確保分析數據能維持一定的正確性。

本專題以python進行資料整理與分析，這篇notebook會按步驟說明「不能安全駕駛」文字清理使用的regex模式說明，並附上酒駕致人於死、酒駕致人受傷的程式碼。


## 引入文字清理使用的模組

- 引入regular expresssion的re模組、數據使用的pandas模組
- 試著讀入我們open data

In [14]:
import re
import pandas as pd

df1 = pd.read_csv('/Volumes/Untitled/opendata/一般酒駕/drunkDrive_2013.csv',sep='\t')
df2 = pd.read_csv('/Volumes/Untitled/opendata/fatal_export.csv',sep='\t')
df3 = pd.read_csv('/Volumes/Untitled/opendata/hurt_export.csv',sep='\t')

df3.head(5)

,Unnamed: 0,主文,地院,年,不能安全駕駛有期徒刑_字串,不能安全駕駛有期徒刑_可計算,不能安全駕駛有期徒刑_文字分組,過失傷害有期徒刑_字串,過失傷害有期徒刑_可計算,過失傷害有期徒刑_文字分組,是否公訴不受理,酒精濃度,是否累犯,車種,2013年前後,2011年前後,判決內容
0,0,主文乙○○駕駛動力交通工具肇事，致人受傷而逃逸，處有期徒刑陸月，如易科罰金，以新臺幣參仟元折...,台北,2007.0,NaN,NaN,無,＊不受理,NaN,公訴不受理,1,1.24,0,小客車,before,before,臺灣臺北地方法院刑事判決公 訴 人 臺灣臺北地方法院檢察署檢察官被告 乙○○上列被告因公共危...
1,1,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金銀元貳萬肆仟元即新臺幣柒萬貳仟元，...,台北,2005.0,NaN,NaN,無,50,50.0,大於1月，2月以下,0,0.76,0,機車,before,before,臺灣臺北地方法院刑事判決 95年度交易緝字第6號公 訴 人 臺灣臺北地方法院檢察署檢察官被告...
2,2,主文丙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣陸萬元，如易服勞役，以新臺幣...,台北,2007.0,NaN,NaN,無,59,59.0,大於1月，2月以下,0,NaN,0,小貨車,before,before,臺灣臺北地方法院刑事判決 96年度交訴字第8號公 訴 人 臺灣臺北地方法院檢察署檢察官被告...
3,3,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金銀元壹萬元即新臺幣參萬元，如易服勞...,台北,2005.0,NaN,NaN,無,20,20.0,1月以下,0,0.47,0,機車,before,before,臺灣臺北地方法院刑事判決 95年度交易字第537號公 訴 人 臺灣臺北地方法院檢察署檢察官被...
4,4,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，科罰金新臺幣壹萬伍仟元，如易服勞役，以新...,台北,2007.0,NaN,NaN,無,＊不符,NaN,其他,0,0.59,0,小客車,before,before,臺灣臺北地方法院刑事判決 95年度交易字第769號公 訴 人 臺灣臺北地方法院檢察署檢察官被...


## 不能安全駕駛切刑期、切罰金

- 判決書全文是非結構化資料，雖然述說同一件事，但是會因不同法官、不同時期，有各種不同的寫法，甚至有錯漏字，因此捕捉特徵的寫法有些複雜，以下會逐步說明。
- 判刑、判罰金都會出現在「判決書主文」中，我們讀入主文的資料進行萃取。至於酒精濃度、車種、教育程度則是從「判決書全文」萃取。
- 萃取時會取出比較大段的部分，再刪去不需要的部分。例如萃取「不能安全駕駛罪」判刑，會先從主文找到「而駕駛動力交通工具，處有期徒刑貳月」，而不直接找「處有期徒刑貳月」的片段，避免取到非不能安全駕駛罪的資料；接著再從中把「而駕駛動力交通工具」、「處有期徒刑」等字刪除。
- 寫regex pattern時，我習慣在 [regular expression101](https://regex101.com/) 網站操作，好處是模式結果可視化，若100筆案例、500筆案例大都可以抓到結果，再回到python批次大量操作。

| ![使用於萃取不能安全駕駛刑期的過程畫面](https://i.imgur.com/AWjj7BN.png =400x) | 
|:--:|
| *當大部分的案例都有色塊時，代表模式可以抓取我們指定的特徵* |


### 不能安全駕駛判決的特徵

要能歸納出模式，必須要花費時間找出判決書寫法的「樣態」，再針對這些樣態進行模式描寫（regular expression pattern）。

例如，酒後駕車在刑法中歸類於不能安全駕駛動力交通工具，查看判決書會發現，有的判決書會提出酒精濃度超過標準、有的只寫不能安全駕駛。經過大量比對後，我們以a1、a2作為法官描述「不能安全駕駛罪」的特徵。

a1是法官提出酒精濃度超過標準，其中包含了幾個寫作用詞的變形，如酒精濃度有吐氣或血液兩種檢測、酒駕標準因年代不同而有零點二五、零點五五等不同標準、標點符號的使用習慣不同、行文的輔助詞（而、之）不同，針對這些詞出現與否進行正規表達式的描寫。

a2則是可以承續酒精濃度的句子，也可以針對比較簡寫的狀況進行抓取。其中融合三種可能描述：
- 甲○○吐氣所含酒精濃度達每公升零點二五毫克以上*__而駕駛動力交通工具__*
- 甲○○服用酒類*__不能安全駕駛動力交通工具而駕駛__*
- 甲○○犯*__不能安全駕駛動力交通工具罪__*

利用a1+a2即可標記出大部分的「不能安全駕駛罪」用詞。

不能安全駕駛罪這20年來有罰金、拘役、有期徒刑等處刑方式，因此以b1鎖定自由刑的用詞、b2鎖定罰金的用詞。以自由刑的b1為例，可以萃取這些語句：
- 甲○○吐氣所含酒精濃度達每公升零點二五毫克以上而駕駛動力交通工具*__，處有期徒刑__*貳月
- 甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，*__處有期刑__*貳月
- 甲○○服用酒類，不能安全駕駛動力交通工具而駕駛*__，處拘役__*伍拾玖日
- 甲○○服用酒類，不能安全駕駛動力交通工具而駕駛*__，累犯，處有期徒刑__*陸月

接著我們再分流，以c1鎖定處刑數量、c2鎖定罰金數量。這部分主要是在處理數字組合與單位（年月日、元）。以罰金的c2為例，可以萃取這些語句：
- 主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣*__玖萬元__*
- 主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，科罰金新臺幣*__柒萬伍仟元__*

綜上所述，我們以a1+a2+b1+c1來萃取不能安全駕駛遭處有期徒刑的語句、以a1+a2+b2+c2來萃取不能安全駕駛遭科罰金的語句。

In [46]:
#鎖定「不能安全駕駛罪」用詞
a1 = "(交通工具(而)*[，,，]*有*(吐氣|血液中)(所含)*酒精濃度達(每公升|百分之)*零點(貳伍|二五|伍伍|五五)(毫克)*以上(之情形)*|"
a2 = "(不能安全駕駛)*(而駕駛)*動力交通工具罪*(而駕駛)*者*)"


#鎖定累犯註記、判刑（拘役、有期徒刑）用詞
b1 = "[，,，](累犯)*(共*罪)*[，,，]*[各均有]*處*[,，]*(有期徒*刑徒*|拘役)"
b2 = "[，,，](累犯)*(共*罪)*[，,，]*[各均有]*[處科]*[,，]*罰金(新[台臺]幣)*"


#鎖定處刑用詞
c1 = "[一二三四五六七八九十壹貳參肆伍陸柒捌玖拾廿卅１２３４５６７８９０]*(年|個*月|日)"
c2 = "[一二三四五六七八九十壹貳參肆伍陸柒捌玖拾廿卅百佰千仟萬１２３４５６７８９０]*元*"


#找出剝奪自由刑之模式組合
findimprison = a1+a2+b1+c1


#找出罰金之模式組合
findpenalty = a1+a2+b2+c2


### 不能安全駕駛無罪部分與取出數字方式

透過findimprison與findpenalty兩種模式可以找出大部分的處刑，但會發現判決書中也有無罪的狀況，因此再增加一種判斷無罪的模式。

當我們有能力從文字中定位單一罪行的片段時，再從中取出我們需要的數字，可以提升精準度。這個過程如下：

1. 【判決主文】乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑參月；又因過失致人於死，處有期徒刑壹年；又駕駛動力交通工具肇事，致人死傷而逃逸，處有期徒刑捌月；應執行有期徒刑壹年陸月。
2. 【鎖定不能安全駕駛】不能安全駕駛動力交通工具而駕駛，處有期徒刑參月
3. 【取出數字】參月

當判斷出處有期徒刑、罰金、無罪，分別切出結果，再把非刑期與罰金的文字都刪除（如罰金、有期徒刑、累犯、標點符號等），就可以得到量刑。以這種方式逐步建立可操作的判決書結構化資料。

In [91]:
#無罪的模式組合
d = "[，,，、，]*部*[分份]*[，,，、，]*均*無罪"
findun = a2+d


#把非刑期、罰金的描述都刪掉（很暴力）
rep = "情形|罰金|科|處|新[台臺]幣|銀|元|不能安全|駕駛|動力|交通工具|而|吐氣|所含|酒精濃度|達|每公升.*毫克|以上|而|者|,|，|、.*|；.*|。.*|累犯|共.罪|有期徒*刑徒*|罪|拘役|過失|如易科.*|被訴.*|應執行.*|緩刑.*|各|均|部[份分].*|共|有|之"


### 實際運作方法

接下來以迴圈方式進行一連串的判斷：
- 是否判有期徒刑，取出量刑
- 是否判罰金，取出量刑
- 是否無罪

In [92]:
result_imprison_text = []
result_penalty_text = []

for i in df1['主文']:
    
    if bool(re.search(findimprison, i)):
        r  = re.sub(rep,"",re.search(findimprison, i).group())
        result_imprison_text.append(r)
        result_penalty_text.append('')
        
    elif bool(re.search(findf,i)):
        r  = re.sub(rep,"",re.search(findf,i).group())
        result_imprison_text.append("")
        result_penalty_text.append(r)
        
    else:
        if bool(re.search(findn,i)):
            result_imprison_text.append("無罪")
            result_penalty_text.append("")
            
        else:
            result_imprison_text.append("")
            result_penalty_text.append("")
            

### 國字數字轉成數字function

抓出刑期與罰金的資訊之後，需將國字換為數字。另外寫成了兩個函式：todays(x) 將刑期國字換為數字、tomoney(x) 將罰金國字換為數字。

步驟大致是：
- 刑期需確定年月日單位，並依一年365日、一月30日進行換算
- 罰金需確定位數
- 將國字取代為數字

In [49]:
def repl(g):
    u = ['壹','貳','參','肆','伍','陸','柒','捌','玖','拾','廿','卅','一','二','三','四','五','六','七','八','九','十',0,'１','２','３','４','５','６','７','８','９','０','佰','仟','萬','百','千']
    p = u.index(g)
    n = [1,2,3,4,5,6,7,8,9,10,20,30,1,2,3,4,5,6,7,8,9,10,0,1,2,3,4,5,6,7,8,9,0,100,1000,10000,100,1000]
    return n[p]

def nds(y):
    if bool(re.search('[壹貳參肆伍陸柒捌玖拾廿卅佰百仟千萬一二三四五六七八九十１２３４５６７８９０]', y)):
        if len(y) == 1:
            return repl(y)

        elif len(y) == 2:
            if y[1]=='拾':
                return repl(y[0])*repl(y[1])

            elif y[0] =='拾' or y[0] =='廿' or y[0]=='卅':
                return repl(y[0])+repl(y[1])

            else:
                return repl(y[0])*10 + repl(y[1])

        elif len(y) == 3:
            return repl(y[0])*repl(y[1])+repl(y[2])

        else:
            return repl(y)
    else:
        return 0

def todays(x):
    if '年' in x or '月' in x or '日' in x:
        try:
            y = x.index("年")
        except:
            y = 0
        try:
            m = x.index("月")
        except:
            m = 0
        try:
            d = x.index("日")
        except:
            d = 0

        if y>0:
            year = x[0:y]

            if m>0:
                month = x[0+y+1:m]

                if d>0:
                    day = x[0+m+1:d]
                else:
                    day = ""
            else:
                month = ""

                if d>0:
                    day = x[0+y+1:d]
                else:
                    day = ""
        else:
            year = ""

            if m>0:
                month = x[0:m]

                if d>0:
                    day = x[0+m+1:d]

                else:
                    day =""
            else:
                month = ""
                day = x[0:d]

        res = nds(year)*365+nds(month)*30+nds(day)
        return res

    else:
        return '＊'+x

def tomoney(x):
    x = x.replace('千','仟').replace('百','佰')
    if '萬' in x or '仟' in x or '佰' in x:
        try:
            w = x.index("萬")
        except:
            w = 0
        try:
            t = x.index("仟")
        except:
            t = 0
        try:
            h = x.index("佰")
        except:
            h = 0

        if w>0:
            wan = x[0:w]

            if t>0:
                tho = x[0+w+1:t]

                if h>0:
                    hun = x[0+t+1:h]
                else:
                    hun = ""
            else:
                tho = ""

                if h>0:
                    hun = x[0+w+1:t]
                else:
                    hun = ""
        else:
            wan = ""

            if t>0:
                tho = x[0:t]

                if h>0:
                    hun = x[0+t+1:h]

                else:
                    hun =""
            else:
                tho = ""
                hun = x[0:h]

        res = nds(wan)*10000+nds(tho)*1000+nds(hun)*100
        return res
    else:
        return '＊'+x

In [94]:
result_imprison_num = []
for j in result_imprison_text:
    try:
        result_imprison_num.append(todays(j))
    except:
        result_imprison_num.append('*'+j)

result_penalty_num = []
for j in result_penalty_text:
    try:
        result_penalty_num.append(tomoney(j))
    except:
        result_penalty_num.append('*'+j)


### 製作成一表格檢查正確與否，是否需要人工修正

In [95]:
dfnew = pd.DataFrame()

dfnew['主文'] = df1['主文']
dfnew['不-刑國字'] = result_imprison_text
dfnew['不-刑'] = result_imprison_num
dfnew['不-罰國字'] = result_penalty_text
dfnew['不-罰'] = result_penalty_num

dfnew

,主文,不-刑國字,不-刑,不-罰國字,不-罰
0,主文黃士桓吐氣所含酒精濃度達每公升零點二五毫克以上而駕駛動力交通工具，處有期徒刑貳月，併科罰...,貳月,60,,＊
1,主文郭庭宏吐氣所含酒精濃度達每公升零點二五毫克以上而駕駛動力交通工具，處有期徒刑貳月，併科罰...,貳月,60,,＊
2,主文陳弘岳吐氣所含酒精濃度達每公升零點二五毫克以上而駕駛動力交通工具，處有期徒刑參月，如易科...,參月,90,,＊
3,主文張名揚服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾玖日，如易科罰金，以新臺幣壹仟...,伍拾玖日,59,,＊
4,主文黃有利服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣陸萬元，如易服勞役，以新臺幣...,,＊,陸萬,60000
5,主文曾昱詮(原名曾敏揚)服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣參萬元，如易服...,,＊,參萬,30000
6,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，累犯，處有期徒刑陸月，併科罰金新臺幣拾貳...,陸月,180,,＊
7,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣陸萬元，如易服勞役，以新臺幣...,,＊,陸萬,60000
8,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣陸萬元，如易服勞役，以新臺幣...,,＊,陸萬,60000
9,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑陸月，如易科罰金，以新臺幣壹仟...,陸月,180,,＊


### 過失致死的模式與切刑期

In [96]:
findimprison = "致人於死[者罪]*[,，](累犯)*(共*罪)*[,，]*[各均]*處*[,，]*(有期徒*刑*徒*|拘役)[一二三四五六七八九十壹貳參肆伍陸柒捌玖拾廿卅１２３４５６７８９０]*(年|個*月|日)*"
findun = "(過失致人於死|過失傷害)[、，]*部*[份分]*[、，]*(均)*無罪"
rep = "致人於死|者|,|，|、.*|；.*|。.*|累犯|處|有期徒*刑徒*|罪|拘役|過失|如易科.*|被訴.*|應執行.*|緩刑.*|各|均|部份.*|共"

result_imprison_text = []
for i in df2['主文']:
    try:
        r  = re.sub(rep, "", re.search(findimprison, i).group())
        result_imprison_text.append(r)
        
    except:
        if bool(re.search(findn, i)):
            result_imprison_text.append("無罪")
        else:
            result_imprison_text.append("")

result_imprison_num = [todays(j) for j in result_imprison_text]

dfnew = pd.DataFrame()
dfnew['主文'] = df2['主文']
dfnew['死-刑國字'] = result_imprison_text
dfnew['死-刑'] = result_imprison_num
dfnew


,主文,死-刑國字,死-刑
0,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛者，處有期徒刑參月，如易科罰金，以銀元參佰...,伍月,150
1,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑參月；又因過失致人於死，處有期...,壹年,365
2,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣參萬元，如易服勞役，以新臺幣...,肆月,120
3,主文丙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑肆月；又因過失致人於死，處有期...,捌月,240
4,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑貳月，如易科罰金，以新臺幣壹仟...,陸月,180
5,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑參月，如易科罰金，以銀元參佰元...,伍月,150
6,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑陸月，減為有期徒刑參月；又因過...,捌月,240
7,主文乙○○因過失致人於死，處有期徒刑肆月，如易科罰金，以新臺幣壹仟元折算壹日；又服用酒類，不...,肆月,120
8,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾日，如易科罰金，以新臺幣壹仟元...,陸月,180
9,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役參拾日，如易科罰金，以新臺幣壹仟元...,陸月,180


### 過失傷害的模式與切刑期

In [97]:
findimprison = "(過失)*傷害而*(人(之身體)*|罪)*者*[,，]*(致人*[重受]傷罪*)*[,，](累犯)*(共貳罪)*[,，]*處*[,，]*(拘役|有期徒*刑徒*)([一二三四五六七八九十壹貳參肆伍陸柒捌玖拾廿卅１２３４５６７８９０]*(年|個*月|日))*"
findun = "(過失)*傷害人*(致人*[重受]傷)*罪*嫌*[、，]*(及.*罪)*(部分|部份)*[、，]*(均)*(無罪|公訴不受理|不受理|不理)"
rep = "(過失)*傷害(人(之身體)*|罪)|者|,|，|、.*|；.*|。.*|累犯|處|有期刑|有期徒刑|罪|拘役|過失|如易科.*|被訴.*|應執行.*|緩刑.*|各|均|部份.*|共|致|人|重|受|傷|而|害"

result_imprison_text = []

for i in df3['主文']:
    try:
        r  = re.sub(rep, "", re.search(findimprison, i).group())
        result_imprison_text.append(r)
    except:
        if bool(re.search(findun, i)):
            if "無罪" in re.search(findun, i).group():
                result_imprison_text.append('無罪')
            elif "不受理" in i:
                result_imprison_text.append("不受理")
        else:
            result_imprison_text.append("*不符")

result_imprison_num = [todays(j) for j in result_imprison_text]

dfnew = pd.DataFrame()
dfnew['主文'] = df3['主文']
dfnew['傷-刑國字'] = result_imprison_text
dfnew['傷-刑'] = result_imprison_num
dfnew


,主文,傷-刑國字,傷-刑
0,主文乙○○駕駛動力交通工具肇事，致人受傷而逃逸，處有期徒刑陸月，如易科罰金，以新臺幣參仟元折...,不受理,＊不受理
1,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金銀元貳萬肆仟元即新臺幣柒萬貳仟元，...,伍拾日,50
2,主文丙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣陸萬元，如易服勞役，以新臺幣...,伍拾玖日,59
3,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金銀元壹萬元即新臺幣參萬元，如易服勞...,貳拾日,20
4,主文乙○○服用酒類，不能安全駕駛動力交通工具而駕駛，科罰金新臺幣壹萬伍仟元，如易服勞役，以新...,*不符,＊*不符
5,主文丙○○服用酒類，不能安全駕駛動力交通工具而駕駛，處有期徒刑貳月，如易科罰金，以新臺幣壹仟...,不受理,＊不受理
6,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役肆拾日，如易科罰金，以銀元參佰元即...,參月,90
7,主文丙○○服用酒類，不能安全駕駛動力交通工具而駕駛，累犯，處有期徒刑拾月；又因過失傷害人，處...,伍月,150
8,主文甲○○攜帶兇器竊盜，累犯，處有期徒刑捌月，減為有期徒刑肆月；扣案六角板手壹支沒收；又服用...,肆月,120
9,主文甲○○服用酒類，不能安全駕駛動力交通工具而駕駛，處罰金新臺幣伍萬元，如易服勞役，以新臺幣...,伍拾日,50


### 補充說明與資料

1. 我們非資訊專業出身，程式能力是透過坊間課程、自學、不斷嘗試錯誤摸索而來，程式撰寫的風格、效率、與精確度等層面都有許多有待改進之處，歡迎方家指教。

2. 這次使用的regular expression由於是處理中文資料，仍在很直觀的層次，大致說明使用到的語法：
    - \* （比對零到多個前一個字），用意多是因行文風格不同在省略部分字詞，如：「...致人於死，累犯，處有期徒刑...」、「...致人於死，處有期徒刑...」這兩種句子，以「致人於死，(累犯)*，處有期徒刑」一種模式就可以比對。
    - [] （比對括號中的其中一個字），用意是在處理行文中相同意思，但不同用字的情況，如：部分、部份；標點符號的全半形等情況。
    - | （或 or），用意是在組合各種可能情況，如：五年、5個月、參拾日，三種單位以「年|個*月|日」一種模式比對。

